In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/OmarAshraf772/OmarAbuzied/raw/refs/heads/main/Projects/depi/deployment.zip

--2024-10-27 06:49:39--  https://github.com/OmarAshraf772/OmarAbuzied/raw/refs/heads/main/Projects/depi/deployment.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/OmarAshraf772/OmarAbuzied/refs/heads/main/Projects/depi/deployment.zip [following]
--2024-10-27 06:49:40--  https://raw.githubusercontent.com/OmarAshraf772/OmarAbuzied/refs/heads/main/Projects/depi/deployment.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11963087 (11M) [application/zip]
Saving to: ‘deployment.zip’

deployment.zip      100%[===================>]  11.41M  71.8MB/s    in 0.2s    

2024-10-27 06:49:40 (71.8 MB/s)

In [ ]:
import zipfile
zip_path = '/content/deployment.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
%%writefile app.py
import zipfile
import pandas as pd
import joblib
import streamlit as st
#from sklearn.preprocessing import StandardScaler

# Load the model, encoder, scaler, and imputer
model = joblib.load('/content/xgb_model (1).pkl')
le = joblib.load('/content/encoders.pkl')
scaler = joblib.load('/content/standard_scaler (2).pkl')

# Ensure these variables are defined
data = pd.read_csv('/content/dataExtracted.csv')
VALID_STATE_ABBREVIATIONS = data['state'].unique().tolist()
VALID_JOBS = data['job'].unique().tolist()
VALID_CATEGORIES = data['category'].unique().tolist()

def categorize_numeric_features(input_df):
    input_df['Age_Category'] = pd.cut(input_df['age'],
                                       bins=[0, 18, 35, 55, float('inf')],
                                       labels=['Child', 'Young Adult', 'Adult', 'Senior'],
                                       right=False)
    input_df['population_cat'] = pd.cut(input_df['population'],
                                         bins=[0, 2500, 20000, 100000, float('inf')],
                                         labels=['Very Small', 'Small', 'Large', 'Very Large'],
                                         right=False)
    return input_df

def preprocess_user_input(user_input):
    input_df = pd.DataFrame([user_input])
    original_input_df = input_df.copy()
    input_df = categorize_numeric_features(input_df)
    input_df.drop(columns=['age', 'population'], inplace=True, errors='ignore')

    for column in ['Hour_Category', 'Displacement_Category', 'state', 'gender', 'job', 'category', 'Age_Category', 'population_cat']:
        if column in le:
            input_df[column] = le[column].transform(input_df[column])

    if 'amt' in input_df.columns:
        input_df[['amt']] = scaler.transform(input_df[['amt']])

    expected_feature_order = ['category', 'amt', 'gender', 'state', 'job',
                              'Hour_Category', 'Age_Category', 'Displacement_Category',
                              'population_cat']
    input_df = input_df.reindex(columns=expected_feature_order)
    return original_input_df, input_df

def make_prediction(user_input):
    original_input_df, processed_input = preprocess_user_input(user_input)
    prediction = model.predict(processed_input)
    prediction_result = "Fraud" if prediction[0] == 1 else "Not Fraud"
    return prediction_result, original_input_df, processed_input

# Define Streamlit application
st.title("Fraud Prediction App")

amt = st.number_input("Transaction Amount")
Hour_Category = st.selectbox("Hour Category", options=['Early Morning', 'Morning', 'Afternoon', 'Evening'])
age = st.number_input("Age")
population = st.number_input("City Population")
Displacement_Category = st.selectbox("Displacement Category", options=['Very Close', 'Close', 'Moderate', 'Far'])
state = st.selectbox("State Abbreviation", options=VALID_STATE_ABBREVIATIONS)
gender = st.selectbox("Gender", options=['M', 'F'])
job = st.selectbox("Job", options=VALID_JOBS)
other_job = st.text_input("If 'Other', please specify:")
category = st.selectbox("Transaction Category", options=VALID_CATEGORIES)

if job == "Other":
    job = other_job

if st.button("Predict"):
    user_input = {
        "category": category,
        "amt": amt,
        "gender": gender,
        "state": state,
        "job": job,
        "Hour_Category": Hour_Category,
        "age": age,
        "Displacement_Category": Displacement_Category,
        "population": population,
    }

    prediction_result, original_input_df, processed_input = make_prediction(user_input)
    st.warning(f"Prediction: {prediction_result}")

    # Save the original user input and processed input to CSV
    user_input_csv_path = "user_input_data.csv"
    processed_input_csv_path = "processed_input_data.csv"

    original_input_df.to_csv(user_input_csv_path, index=False)
    processed_input.to_csv(processed_input_csv_path, index=False)

    # Provide download buttons for the CSV files
    #with open(user_input_csv_path, "rb") as f:
     #   st.download_button(label="Download User Input CSV", data=f, file_name="user_input_data.csv")

    #with open(processed_input_csv_path, "rb") as f:
     #   st.download_button(label="Download Processed Input CSV", data=f, file_name="processed_input_data.csv")

Overwriting app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.229.49.76


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.49.76:8501

your url is: https://fresh-results-prove.loca.lt
  Stopping...
^C
